In [1]:
%cd ..
%load_ext nb_black

/home/hardianlawi/attention-lstm-sentiment-analysis


<IPython.core.display.Javascript object>

In [2]:
import json
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from src.preprocess import Preprocessor
from src.data import get_data
from src.models import get_model

tf.random.set_seed(2020)

vocab_size = 5000
emb_size = 32
max_words = 500
oov_token = "<OOV>"

(str_X_train, y_train), (str_X_val, y_val), (str_X_test, y_test) = get_data()

preprocessor = Preprocessor(
    maxlen=max_words, vocab_size=vocab_size, oov_token=oov_token
)
preprocessor.fit_on_texts(str_X_train + str_X_val + str_X_test)

X_train = preprocessor.transform(str_X_train)
X_val = preprocessor.transform(str_X_val)
X_test = preprocessor.transform(str_X_test)

<IPython.core.display.Javascript object>

In [3]:
def generate_scores(i):

    models = []
    num_params = []
    accs = []

    def train(hs, ahs):

        model = get_model(
            "attention",
            max_words,
            vocab_size,
            emb_size,
            hidden_size=hs,
            attention_hs=ahs,
        )
        model.compile(
            loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
        )

        batch_size = 128
        num_epochs = 3

        model.fit(
            X_train,
            y_train,
            validation_data=(X_val, y_val),
            batch_size=batch_size,
            epochs=num_epochs,
        )

        scores = model.evaluate(X_test, y_test, verbose=0)
        print("Test accuracy:", scores[1])

        return scores[1], model.count_params(), model

    hparams = [(32, 16), (64, 16)]
    for (hs, ahs) in hparams:

        print(f"{(hs, ahs)}")

        acc, num_param, model = train(hs, ahs)
        accs.append(float(acc))
        num_params.append(num_param)
        models.append(model)

    perfs = {
        "hparams": hparams,
        "num_params": [int(x) for x in num_params],
        "accs": [float(x) for x in accs],
    }

    with open(f"attention_perfs_{i}.json", "w") as f:
        json.dump(perfs, f)


for i in range(3):
    generate_scores(i)

(32, 16)
Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================] - 89s 4ms/sample - loss: 0.5239 - accuracy: 0.7496 - val_loss: 0.3253 - val_accuracy: 0.8664
Epoch 2/3
23750/23750 [==============================] - 85s 4ms/sample - loss: 0.2787 - accuracy: 0.8902 - val_loss: 0.2701 - val_accuracy: 0.8848
Epoch 3/3
23750/23750 [==============================] - 82s 3ms/sample - loss: 0.2254 - accuracy: 0.9149 - val_loss: 0.2605 - val_accuracy: 0.8904
Test accuracy: 0.87756
(64, 16)
Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================] - 124s 5ms/sample - loss: 0.4966 - accuracy: 0.7586 - val_loss: 0.3056 - val_accuracy: 0.8712
Epoch 2/3
23750/23750 [==============================] - 120s 5ms/sample - loss: 0.2632 - accuracy: 0.8973 - val_loss: 0.2794 - val_accuracy: 0.8944
Epoch 3/3
23750/23750 [==============================] - 117s 5ms/sample - loss: 0.2192 - accuracy: 0.9163 - val_loss: 

<IPython.core.display.Javascript object>